In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error, mean_absolute_error


In [2]:
pjme = pd.read_csv('/Users/sean/Downloads/energy_forecast/PJME_hourly.csv',
                  index_col=[0], parse_dates=[0])

In [4]:
import tensorflow as tf

print(tf.__version__)

2.3.1


In [43]:
pjme.head()

def extract_feats(df): 
    df['date'] = df.index.day
    df['month']  = df.index.month
    df['hour'] = df.index.hour
    df['day_of_year'] = df.index.dayofyear
    df['day_of_week'] = df.index.dayofweek
    df['week_of_year'] = df.index.weekofyear
    return df

In [50]:
X_dense = extract_feats(pjme)

/anaconda3/envs/ee127/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  if __name__ == '__main__':


In [56]:
X_dense.head()

,PJME_MW,date,month,hour,day_of_year,day_of_week,week_of_year
Datetime,,,,,,,
2002-12-31 01:00:00,26498.0,31,12,1,365,1,1
2002-12-31 02:00:00,25147.0,31,12,2,365,1,1
2002-12-31 03:00:00,24574.0,31,12,3,365,1,1
2002-12-31 04:00:00,24393.0,31,12,4,365,1,1
2002-12-31 05:00:00,24860.0,31,12,5,365,1,1


In [57]:
X_dense = X_dense.iloc[:, 1:].values

X_dense.shape

(145366, 6)

In [61]:
def onehot(d): 
    """
    takea feature vector and make it one hot. 
    """
    print(list(set(d)))
    result = np.zeros((len(d), len(list(set(d)))))
    for k, dd in enumerate(d):
#         print(k, dd)
        result[k, (int(dd)-1)] = 1
        
    return result

def to_onehot(X, feat_ind=-1): 
    """
    makes the variable at feat_ind into a onehot encoding. 
    """
    og_feat = X[:, feat_ind]
    onehot_feat = onehot(og_feat)
    new_X = np.concatenate((X, onehot_feat), axis=-1)
    return new_X

In [62]:
X_dense = to_onehot(X_dense)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53]


In [83]:


def tf_windowing(X, window_sz, bs=1000):
    dataset = tf.data.Dataset.from_tensor_slices(X)
    dataset = dataset.window(size=window_sz+1, shift=1,drop_remainder=True)
    dataset = dataset.flat_map(lambda x: x.batch(window_sz+1))
    dataset = dataset.shuffle(bs).map(lambda x: (x[:window_sz], x[window_sz:]))
    dataset = dataset.batch(batch_size=32).prefetch(1)
    
    return dataset
    
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:-1], window[-1]))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

In [87]:
window_sz = 2
feats = 1

dataset = tf_windowing(X, window_sz=window_sz)

dataset = windowed_dataset(X, window_size=window_sz, batch_size=32, shuffle_buffer=1000)

for x,y in dataset: 
    print(x.shape, y.shape)

(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (

(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (

(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (

(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (

(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)
(32, 2, 1) (32, 1)


KeyboardInterrupt: 

In [94]:
a = tf.keras.layers.Input(shape=(window_sz, feats))
x = tf.keras.layers.Flatten()(a)
x = tf.keras.layers.Dense(20, activation='relu')(x)
x = tf.keras.layers.Dense(20, activation='relu')(x)
x = tf.keras.layers.Dense(1)(x)
y = tf.keras.layers.Lambda(lambda x: x/1000)(x)

model = tf.keras.Model(a, y)

In [95]:
model.compile(loss='mse', 
             optimizer='adam')

In [96]:
model.fit(dataset, verbose=True, epochs=3)

Epoch 1/3
   2136/Unknown - 4s 2ms/step - loss: 305161952.0000

KeyboardInterrupt: 